# Camera motion estimation

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
vid_path = "/Users/senneloobuyck/Library/CloudStorage/OneDrive-UGent/Documenten/UGent_2022-2023/VENTOUX_v1.0/01_inputs/videos/2020-Kuurne-Brussel-Kuurne.mp4"

In [3]:
cap = cv2.VideoCapture(vid_path)

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
qualityLevel = 0.3,
minDistance = 7,
blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize = (15, 15),
maxLevel = 2,
criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# Renew good features every 10 frames
feature_refresh_rate = 3

# Counter
counter = 1

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
    ret, frame = cap.read()

    if not ret:
        print('No frames grabbed!')
        break
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if counter % feature_refresh_rate == 0:
        p0 = cv2.goodFeaturesToTrack(frame_gray, mask = None, **feature_params)
        print(f"Refreshing features, p0 = {p0}")

    # Calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]

    # Draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
        img = cv2.add(frame, mask)
        
        cv2.imshow('frame', img)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
    # Calculate average motion vectors
    avg_motion_x = -(np.mean(good_new[:, 0] - good_old[:, 0]))
    avg_motion_y = -(np.mean(good_new[:, 1] - good_old[:, 1]))
    print('Average motion vector: ({:.2f}, {:.2f})'.format(avg_motion_x, avg_motion_y))

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

    counter += 1

    cv2.destroyAllWindows()

OpenCV: Couldn't read video stream from file "/Users/senneloobuyck/Library/CloudStorage/OneDrive-UGent/Documenten/UGent_2022-2023/VENTOUX_v1.0/01_inputs/videos/2020-Kuurne-Brussel-Kuurne.mp4"


error: OpenCV(4.8.0) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
